# Run time

Tests complete in slightly less than 1 minute on a single core.

# Purpose

Pyiron accepts simulation cells of any arbitrary shape,

```
[[ax, ay, az],
 [bx, by, bz],
 [cx, cy, cz]]
```

In contrast, Lammps accepts cells which are formatted only a certain way. Namely,

```
[[lx,  0,  0],
 [xy, ly,  0],
 [xz, yz, lz]]
 ```
 
This format is still sufficiently general to represent any cell, but it is important to accurately convert our cell (and positions etc) to the Lammps coordinate frame, and then to transform back when we are reading the Lammps output.

This notebook is designed to test that the evolution of cell shapes and sizes functions properly by checking the following things:

- By deforming several representations of the bulk system (primitive, orthorhombic, and cubic) then minimizing them, do they all return the same energy and volume per atom in their minimized state?
- Running zero pressure MD on the primitive cell, then using the final structure output to start a new static calculation, do all the outputs from the final step of the MD match with the static calculation?
- Do the volumes dumped by Lammps agree with the volumes in our cell output?
- Lammps accepts only right-handed coordinate systems, do we comply with this?

These tests are run for both interactive and regular Lammps jobs, and for representitives of all three basic crystal structures (FCC, BCC, and HCP).

In [ ]:
from pyiron.project import Project 
import numpy as np

In [ ]:
pr = Project('integration_test_Lammps_cell_shape_changes')
pr.remove_jobs(recursive=True)

In [ ]:
interactive_options = [False, True] 
hosts = ['Al', 'Mo', 'Mg']
cell_modes = ['cubic', 'orthorhombic', 'primitive']

In [ ]:
def make_unit_cell(host, cell_mode):
    if cell_mode == 'cubic':
        return pr.create_ase_bulk(host, cubic=True)
    elif cell_mode == 'orthorhombic':
        return pr.create_ase_bulk(host, orthorhombic=True)
    else:
        return pr.create_ase_bulk(host)

In [ ]:
cell_deformation = np.array([
    [ 0.0, 0.0,  0.8],
    [-0.2, 0.0, -0.1],
    [ 0.0, 0.5,  0.0]
])
# Be a little bit careful here; for deformations that are too big, the minimizer can fail
# and stop early, leading to a failure that isn't due to anything bad in our code, but 
# just pushing the Lammps minimizer too far.

f_tol = 1e-8
max_iter = 10000
std_dev_tolerance = 1e-4
n_print = max_iter

jobs = []

for is_interactive in interactive_options:
    for host in hosts:
        per_atom_energies = []
        per_atom_volumes = []
        
        for cell_mode in cell_modes:
            structure = make_unit_cell(host, cell_mode)   
            structure.cell += cell_deformation  # Deform cell so the relaxation is interesting/necessary
            
            job_name = '_'.join([host, str(is_interactive), cell_mode, 'minimization'])
            job = pr.create_job(pr.job_type.Lammps, job_name)
            job.structure = structure
            job.potential = job.list_potentials()[0]
            job.calc_minimize(pressure=6*[0.], f_tol=f_tol, max_iter=max_iter, n_print=n_print)
            job.run()
            per_atom_energies.append(job.output.energy_pot[-1] / len(structure))
            per_atom_volumes.append(job.output.volume[-1] / len(structure))
            jobs.append(job)
        
        assert np.std(per_atom_energies) < std_dev_tolerance, "Energies among cell types not equivalent"
        assert np.std(per_atom_volumes) < std_dev_tolerance, "Volumes among cell types not equivalent"
            

In [ ]:
for is_interactive in interactive_options:
    for host in hosts:
        for cell_mode in cell_modes:
            structure = make_unit_cell(host, cell_mode)
            
            job_md_name = '_'.join([host, str(is_interactive), cell_mode, 'md'])
            job_md = pr.create_job(pr.job_type.Lammps, job_md_name)
            job_md.structure = structure.repeat(2)
            job_md.potential = job_md.list_potentials()[0]
            job_md.calc_md(temperature=300, pressure=6*[0], n_ionic_steps=100, n_print=10)
            job_md.run()
            assert(np.allclose(
                job_md.output.volume, 
                [np.linalg.det(cell) for cell in job_md['output/generic/cells']]
            ))
            
            job_static_name = '_'.join([host, str(is_interactive), cell_mode, 'static'])
            job_static = pr.create_job(pr.job_type.Lammps, job_static_name)
            job_static.structure = job_md.get_structure(iteration_step=-1).copy()
            job_static.potential = job_static.list_potentials()[0]
            job_static.calc_static()
            job_static.run()
            
            assert np.allclose(job_md.output.cells[-1], job_static.output.cells[-1]), "Cells differ"
            assert np.isclose(job_md.output.energy_pot[-1], job_static.output.energy_pot[-1]), "Energies differ"
            assert np.allclose(job_md.output.forces[-1], job_static.output.forces[-1]), "Forces differ"
            
            job_md_struct = job_md.get_structure(iteration_step=-1).copy()
            job_md_struct.wrap()
            job_static_struct = job_static.get_structure(iteration_step=-1).copy()
            job_static_struct.wrap()
            assert np.allclose(job_md_struct.positions, job_static_struct.positions), "Wrapped positions differ"
            
            job_md_struct.positions = job_md.output.unwrapped_positions[-1]
            job_md_struct.wrap()
            job_static_struct.positions = job_static.output.unwrapped_positions[-1]
            job_static_struct.wrap()
            assert np.allclose(job_md_struct.positions, job_static_struct.positions), \
                "Wrapped unwrapped positions differ"

In [ ]:
# Clean up
pr.remove_jobs(recursive=True)
pr.remove(enable=True)